# Cell Segmentation for Electron Microscopy - Mitochondria

This is an example notebook for creating segmentation masks of cell organelles in EM images using a fine-tuned AI segmentation model. The model is based on YOLOv5m-seg, which has been trained on a set of EM data including labels for two categories: mitochondria and fat-lipids. 

The trained model weights are published on the Github repo in [yolov5m_seg_mito_lipids.pt](https://github.com/Sydney-Informatics-Hub/PIPE-4011-EM-CV/blob/main/yoloseg/models/yolov5m_seg_mito_lipids.pt).

The trained model's accuracy (mAP-50 score) is:
- mitochondria: 0.86
- fat lipids: 0.92

## Setup

Clone GitHub [repository](https://github.com/Sydney-Informatics-Hub/PIPE-4011-EM-CV) and install model dependencies.

In [ ]:
# clone github if not done yet
!git clone https://github.com/Sydney-Informatics-Hub/PIPE-4011-EM-CV  

# install requirements
%cd PIPE-4011-EM-CV/yoloseg
%pip install -qr requirements.txt

In [ ]:
import sys
import os
import torch
import utils
import subprocess
display = utils.notebook_init()  # checks

In [ ]:
## Add tool modules to path for pre and post processing
sys.path.append('../tools')
import yolo2mask
import image2tiles
import merge_tiles

## Settings

Define input and output settings. All data including model weights and test-data is included in the Github repo.
The tile size is dependent on the input image size (here testdata has 8kx8k pixels) and the size of object of interest. 

In [ ]:
# Path to input images
path_images = '../testdata/sample_highres'

# Path to results
path_results = '../results_mito'

# Path to the trained model
path_model = 'models/yolov5m_seg_mito_lipids.pt'

# tile sizes for inference (dependent on input image size and size of objects of interest)
tile_size = 1280

## Prepare Data

Slice images into smaller tiles. This will help for the model inference.

In [ ]:
# List of images to process
accepted_formats = ["png", "jpg", "jpeg", "tif", "tiff"]
image_list = [f for f in os.listdir(path_images) if f.split(".")[-1].lower() in accepted_formats]
basenames = [f.split(".")[0] for f in image_list]
print("Found {} images to process.".format(len(image_list)))

In [ ]:
path_tiles = os.path.join(path_results, 'tiles')
image2tiles.tile_images_in_folder(path_images, tile_width=tile_size, tile_height=tile_size, max_overlap_ratio=0, outpath=path_tiles)

## Segmentation Predictions

Use finetuned YOLO segmentation model to generate segmentation labels. You can adjust the confidence threshold (number behind `conf` argument below) to include detections at lower or higher confidence level (range: 0 to 1).

In [ ]:
outpat_pred = os.path.join(path_results, "predict-seg")
cmd = f"python predict.py --weights {path_model} --source {path_tiles} --project {outpat_pred} --img 640 --conf 0.4 --line-thickness 3 --save-txt"
subprocess.run(cmd.split())

## Generate image masks

Convert YOLO output labels and segmentation polygons (.txt files) to image binary masks (jpg images). Here cat_id=0 is applied to show only segmentation for the category mitochondria. If you like to show segmentation for all categories (for model here mitochondia and fat lipids), remove cat_id argument below or set to None.

In [ ]:
path_labels = os.path.join(outpat_pred, 'exp/labels')
path_masks = os.path.join(path_results, 'masks')
yolo2mask.convert_to_mask(path_labels, path_tiles, outpath = path_masks, cat_id=0, format_out="jpg")

## Merge masks

Merge mask tiles back to fullsize image at original image size.

In [ ]:
outpath_merged = os.path.join(path_results, 'masks_merged')
os.makedirs(outpath_merged, exist_ok=True)
for basename in basenames:
    merge_tiles.merge(path_masks, basename, output_file= os.path.join(outpath_merged, f'{basename}_mask_mito.jpg'))

## Finish

Inspect the results in the following result subfolders:
- `predict-seg`: prediction contours for tiles with mitochondria and fat lipids
- `masks`: tile binary masks for mitochondria
- `masks_merged`: merged bindary mask for mitochondria at same image size as input image